In [ ]:
import os
!pip install PyPDF2
import PyPDF2
import google.generativeai as genai
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def extract_text_from_pdf(pdf_path):
    """Extract text content from a PDF file."""
    text = ""
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            for page in pdf_reader.pages:
                text += page.extract_text()
        return text
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return None

def extract_skills(text, api_key):
    """Extract skills from text using the Generative AI model."""
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('gemini-pro')

    prompt = f"""
    Extract and list the key skills and requirements from the following text:

    {text}

    Format the output as a Python list of strings.
    """
    try:
        response = model.generate_content(prompt)
        skills = eval(response.text)  # Convert string representation to a list
        return skills
    except Exception as e:
        print(f"Error extracting skills: {e}")
        return []

def analyze_job_description(job_description, resume_text, api_key):
    """Analyze a job description and compare it to the resume."""
    # Extract skills from job description and resume
    job_skills = extract_skills(job_description, api_key)
    resume_skills = extract_skills(resume_text, api_key)

    # Calculate similarity score
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([' '.join(job_skills), ' '.join(resume_skills)])
    similarity_score = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

    # Identify missing skills
    missing_skills = list(set(job_skills) - set(resume_skills))

    return {
        'similarity_score': similarity_score * 100,  # Convert to percentage
        'key_skills': job_skills,
        'missing_skills': missing_skills
    }


def main():
    # Get Google API key
    api_key = 'your_api_key'
    if not api_key:
        print("You can get a free Google API key from: https://makersuite.google.com/app/apikey")
        api_key = 'your_google_api_key'

    # Get PDF path
    pdf_path = '/content/NithinS-Resume.pdf'

    # Extract text from PDF
    resume_text = extract_text_from_pdf(pdf_path)
    if not resume_text:
        print("Failed to extract text from PDF")
        return

    # Analyze job description
    job_description = """
    We are looking for a Python developer with experience in Django and REST APIs.
    The ideal candidate should have knowledge of machine learning libraries like TensorFlow or PyTorch.
    Experience with cloud platforms (AWS, GCP, or Azure) is a plus.
    """
    job_analysis = analyze_job_description(job_description, resume_text, api_key)

    # Save analyses to files

    with open('job_analysis.txt', 'w') as f:
        f.write(str(job_analysis))

    print("\nJob description analysis has been generated and saved to 'job_analysis.txt'")
    print("\nJob Description Analysis:")
    print("-" * 50)
    print(job_analysis)

if __name__ == "__main__":
    main()
